# ECE 594N HW3, Ryan Guajardo, 8619710

# I. Introduction (Markdown)

##### Set the context. What problem is this paper trying to solve? Why is this important?

The paper I'm covering is "Kernel Methods on the Riemannian Manifold of Symmetric Positive Definite Matrices". They begin the paper by explaining that Symmetric Positive Definite (SPD) matrices are popular for encoding image information and that accounting for the Riemannian manifold of SPD matrices allows many algorithms to be used. They then explain that there is issues with approximating the shape of the manifold locally by using the tangent plane of it, mainly that this is only locally viable and doesnt truly represent the true shape of the manifold. Additionally tangent space mapping is highly computationally intensive because logarithmic and exponential maps are used to iteratively map points from the manifold to the tangent space and back again. They then propose that the solution to this problem lies in mapping SPD matrices to a high dimensional Hilbert space where they can then apply Euclidean geometry. In order to do this they introduce a family of positive definite kernels  on the Reimannian manifold of SPD matrices, which are derived from the Gaussian RBF (Radial Basis Function) but augmented with different notions of distances owed to the Log-Euclidean metric. In the end this is valuable because it allows the use of kernel based algorithms developed for standard Euclidean space to be used, while accounting for the geometry of the Riemannian manifold of SPD matrices.

##### Brief literature review: Identify 3+ related papers/models and briefly explain how they relate to one another (1 - 2 sentences per paper).

"Fast an Simple Computations on Tensors with Log-Euclidean Metrics":
This paper presents a new family of metrics called Log-Euclidian, the reason this is pertinent to the main paper is because we use the Log-Euclidean metric to represent a true geodesic distance measurement on SPD. We use this true geodesic distance to define an RBF kernel augmented with the Log-Eudclidean distance measurement used throughout the paper. 

"Region Covariance: A Fast Descriptor for Detection and Classification":
This paper goes over the method to calculate the region covariance which is used to combine feature vectors in all of the examples shown in the main paper. I had to understand this paper in order to be able to implement the algorithm that I chose to replicate. In this case it was the Texture Classification algorithm. 

"Filtering for Texture Classification: A Comparative Study": This paper introduces the Brodatz dataset and shows ways to extract features from texture images.

"Kernel Analysis over Riemannian Manifolds for Visual Recognition of Actions, Pedestrians and Textures": In this paper they outline the pitfalls of using tangent spaces to embed Reimannian manifolds in euclidean space. They say tangent spaces only represent "true" geodesic distances in certain cases and propose instead to embed the manifold into a Reproducing Kernel Hilbert Space by introducing a Riemannian pseudo kernel. This is pertinent because it is exactly what is proposed in the  main paper I cover, also many of their experiments are very similar. 



# II. Background and Model (Markdown)

##### Introduce and explain the model and any necessary mathematical background. Use LaTeX for equations.

The main idea is that we want to use a kernel to embed the manifold in a RKHS, where we can then apply generalized Euclidean algorithms. According to Mercer's theorem this kernel must be positive definite. The Gaussian kernel is a PD kernel in $R^n$. They propose that replacing the Euclidean distance in the Gaussian Kernel with the geodesic distance on the manifold will do this but it must be positive definite. We are working in the space of $dxd$ SPD matrices, and the metric that is proposed is the log-Euclidean metric, which is shown to be a true representation of the geodesic distance on $Sym_d^+$.

##### In general kernel methods can be generalized to non-manifolds like so:
Each point x on a non-linear manifold $M$ is mapped to a feature vector $φ(x)$ in a Hilbert space $H$, the Cauchy completion of the space spanned by real-valued functions defined on $M$. A kernel function $k : (M×M) → R$ is used to define the inner product on H, thus making it a Reproducing Kernel Hilbert Space (RKHS). According to Mercer’s theorem, however, only positive definite kernels define valid RKHS.

##### Gaussian Radial Basis Function
The Gaussian radial basis function (RBF) maps data points to an infinite dimensional Hilbert space. In $R^n$ the Gaussian kernel is $k_G(x_i,x_j):=exp(\frac{||x_i-x_j||^2}{2\sigma^2})$. Here the Euclidian distance measure is used. To define this on the Reimannian manifold we use the notion of log-Euclidean Distance defined as $d_g = ||log(X_i)-log(X_j)||_F$ where $||.||_F$ is the frobenius norm induced by the Frobenius inner product. In this way we can accuratley measure the true geodesic distance on $Sym_d^+$ Overall our new kernel can then be written: $k_R(X_i,X_j):=exp(-\frac{||log(X_i)-log(X_j)||_F^2}{2\sigma^2})$ and it is also defined as positive definite following Theorem 4.1 in the paper. 

"Theorem 4.1. Let (M, d) be a metric space and define $k : (M × M) → R$ by $k(x_i,x_j) := exp(−d^2(x_i,x_j)/2σ2)$. Then, $k$ is a positive definite kernel for all $σ > 0$ if and only if there exists an inner product space $V$ and a function $ψ:M→V$ such that, $d(xi,xj)=∥ψ(xi)−ψ(xj)∥V$. "


##### Kernel PCA on $Sym_d^+$
All points $X_i$ are mapped to feature vectors $H$ giving us a transformed set ${\phi(X_i)}_{i=1}^m$. We then calculate the covariance matrix of the transformed set. This is equivalent to computing the kernel matrix of the original data using $k_R$, the Reimannian Kernel. We then compute the eigenvectors of the kernel matrix. This boils down to a Euclidean representation of the manifold valued data while accounting for the actual geometry of $Sym_d^+$ 



# III. Implementation (Code) 

For the implementation portion of the assignment you can look at the Demonstration and Analysis portion where I attempt to replicate the results for 6.3 Texture Recognition using kernel PCA on $Sym_5^+$ with a  Riemannian kernel. The Implementation of their kernel is shown in the section where we define the gram matrix, that section is titled "define Riemannian kernel for kernel PCA using geomstats". Here you can see my implementation of their kernel using geomstats elementary operations. 

# IV. Demonstration and Analysis (Code / Markdown)

Shown below is the implementation of "6.3 Texture Recognition" in the paper. The pipeline is.
1. Load Brodatz Dataset only 512x512, no Histogram Equalization
2. Split each image into 4, 2 for train, 2 for test.
3. For every test and train image generate the 5 dimensional feature vectors.
4. for each training image, generate 50 random windows 128x128 and compute the covariance descriptors originating from the 5 dim feature vectors.
4. for each test image, generate 100 random windows 128x128 and compute the covariance descriptors originating from the 5 dim feature vectors.
5. Generate Gram Matrix for using our custom kernel, this is necessary when using a precomputed kernel in Kernel PCA.
6. use Kernel PCA on $Sym_5^+$ with the Riemannian Kernel to extract l principle directions and project the training data there
6. (part b ) figure out the best sigma value, by cross validation.
7. do the same with the test data 
8. Decide the class of the test data using k=5 nearest neighbor classifier
9. Compare to Euclidean Distance Metric

Results for this (accuracy and confusion matrix) are shown at the very end in the section titled "using smallest good sigma value, sigma = 0.4, varying L, Riemannian Metric" and "using smallest good sigma value, sigma = 0.4, varying L, Euclidean Metric." 

Here you can see the difference between the Riemannian Metric and the Euclidean Metric similar to what is shown in the paper. The Riemannian Metric performed very well garnering over 90% in all dimensionalities L = [10,11,12,15]. Although it performed well, the amount of computation time was actually quite long, especially when performing the gram matrix computation. I believe in one of the papers outlining the region covariance, "Region Covariance: A Fast Descriptor for Detection and Classification", they outline a faster way to do computations, however I did not have time to implement this. 

In comparison the RBF with Euclidean Metric in place did very poorly, although it was fast. I'm not sure if this is an error? I would assume it is, due to the accuracy measures in the paper being around 90% for all L-dimensional euclidean spaces, but I also understand that implementions will always be different and so I may have not performed the algorithm or pre-processing exactly as they may have. Even performing fitting on the training set it looks like it is struggling to classify points it has already seen, the only thing that seems to help the Euclidean Metric is increasing dimensionality as it gains some ground there and starts to do okay. I tried many fixes and nothing seemed to help aside from increasing the dimesionality. I also tried a standard PCA,(which performed rather well) but that isn't what is outlined in the paper and so it seemed irrelevant. 



In [ ]:
from skimage.filters import threshold_otsu
import numpy as np
from scipy import ndimage
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from skimage import color
from skimage.filters import threshold_otsu, sobel, gaussian
matplotlib.rcParams['figure.figsize'] = (20, 8)
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
%pip install opencv-contrib-python
from PIL import Image
import cv2
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
! pip install geomstats
import geomstats.backend as gs
!pip install -U matplotlib


# Set Up Functions


## divide images into 4

In [2]:
def divide_img(img):
  x = [0,256]
  y = [0,256]
  train1 = img[0+x[0]:256+x[0],0+y[0]:256+y[0]]
  train2 = img[0+x[0]:256+x[0],0+y[1]:256+y[1]]
  test1 = img[0+x[1]:256+x[1],0+y[0]:256+y[0]]
  test2 = img[0+x[1]:256+x[1],0+y[1]:256+y[1]]
  return train1, train2, test1, test2

## Feature Extraction, for Covariance Descriptor

In [3]:
def get_feature_space(img):
  '''
  maps each pixel to a 5 dimensional feature space as in
  "Region Covariance: A Fast Descriptor for Detection and Classification"
  '''
  from numpy import linalg as LA

  kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
  kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])

  I_x = cv2.filter2D(img,cv2.CV_8U,kernelx)
  I_y = cv2.filter2D(img,cv2.CV_8U,kernely)

  I_xx = cv2.filter2D(I_x,cv2.CV_8U,kernelx)
  I_yy = cv2.filter2D(I_y,cv2.CV_8U,kernely)

  F = [img, I_x, I_y, I_xx, I_yy]

  return np.array(F)

In [4]:
def get_features(img):
  from numpy import linalg as LA
  kernely = np.array([[1,1,1],[0,0,0],[-1,-1,-1]])
  kernelx = np.array([[1,0,-1],[1,0,-1],[1,0,-1]])

  I_x = cv2.filter2D(img,cv2.CV_8U,kernelx)
  I_y = cv2.filter2D(img,cv2.CV_8U,kernely)

  I_xx = cv2.filter2D(I_x,cv2.CV_8U,kernelx)
  I_yy = cv2.filter2D(I_y,cv2.CV_8U,kernely)

  I = np.mean(img)

  return [I, LA.norm(I_x,'fro'), LA.norm(I_y,'fro'), LA.norm(I_xx,'fro'), LA.norm(I_yy,'fro')]


## get a random window


In [5]:
def get_random_window(img):
  '''
  returns a random window with size 128x128 from our input image
  '''
  import random
  window_size = 128
  x = random.randrange(0,128,1)
  y = random.randrange(0,128,1)
  return img[x:x+window_size,y:y+window_size]

## Calculate Covariance Descriptors


In [6]:
def get_means(F):
  u = []
  for i in range(len(F)):
    u.append( np.sum(F[i]) / (len(F[i])*len(F[i])) )
  return np.array(u)

In [7]:
def zk(x,y,F):
  '''
  calculate zk for the given index,
  zk is a 5 d vector for the given point
  '''
  zk = []
  for i in range(len(F)):
    zk.append(F[i][x][y])

  return np.array(zk)


In [8]:
def get_covariance_descriptor(F):
  '''
  get covariance descriptor matrix for particular F calculated with a window
  '''
  u = get_means(F)
  C = np.zeros((5,5))

  for i in range(F[0].shape[0]):
    for j in range(F[0].shape[0]):
      a = zk(i,j,F)-u
      C = C+np.dot(a[:,None],a[None,:])

  return C/(len(F[0])*len(F[0]))


# Setup Data

In [9]:
path = '/content/drive/MyDrive/ECE594N/hw3/textures/'

## generate paths

In [10]:
# generating paths
im_paths = []
for j in range(2):
  for i in range(13):
    if (i+1)<10:
      num = '1.'+str(j+1)+'.0'+str(i+1)+'.tiff'
      im_paths.append(num)
      # print(numbers)
    else:
      num = str(i+1)
      num = '1.'+str(j+1)+'.'+str(i+1)+'.tiff'
      im_paths.append(num)
      # print(numbers)

print(im_paths)

['1.1.01.tiff', '1.1.02.tiff', '1.1.03.tiff', '1.1.04.tiff', '1.1.05.tiff', '1.1.06.tiff', '1.1.07.tiff', '1.1.08.tiff', '1.1.09.tiff', '1.1.10.tiff', '1.1.11.tiff', '1.1.12.tiff', '1.1.13.tiff', '1.2.01.tiff', '1.2.02.tiff', '1.2.03.tiff', '1.2.04.tiff', '1.2.05.tiff', '1.2.06.tiff', '1.2.07.tiff', '1.2.08.tiff', '1.2.09.tiff', '1.2.10.tiff', '1.2.11.tiff', '1.2.12.tiff', '1.2.13.tiff']


## labels for the textures

In [11]:
#labels for each texture in order
label = ['Grass',
          'Bark',
          'Straw',
          'Herringbone weave',
          'Woolen cloth',
          'Pressed calf leather',
          'Beach sand',
          'Water',
          'Wood grain',
          'Raffia',
          'Pigskin',
          'Brick wall',
          'Plastic bubbles']

## put everything into dataframe

In [138]:
data = pd.DataFrame()
images = []
paths = []
labels = []

for i in range(len(im_paths)):

  img = cv2.imread(path+im_paths[i])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to grayscale

  paths.append(path+im_paths[i])
  images.append(img)

  #restart index for labels after 13
  if i+1>13:
    labels.append(label[i-13])
  else:
    labels.append(label[i])
  
data['paths'] = paths
data['images'] = images
data['labels'] = labels

In [149]:
data

,paths,images,labels
0,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[27, 30, 39, 59, 180, 115, 199, 174, 124, 93,...",Grass
1,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[166, 126, 135, 143, 111, 69, 13, 25, 58, 130...",Bark
2,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[164, 188, 201, 186, 170, 157, 141, 151, 133,...",Straw
3,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[219, 192, 203, 170, 166, 133, 150, 165, 162,...",Herringbone weave
4,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[168, 157, 157, 179, 201, 195, 191, 173, 185,...",Woolen cloth
5,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[207, 88, 81, 91, 99, 163, 197, 178, 130, 131...",Pressed calf leather
6,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[161, 174, 167, 156, 191, 202, 194, 153, 137,...",Beach sand
7,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[156, 163, 168, 161, 126, 145, 167, 171, 156,...",Water
8,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[189, 199, 201, 195, 184, 199, 201, 197, 198,...",Wood grain
9,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[150, 182, 188, 187, 184, 180, 185, 195, 168,...",Raffia


## augment dataframe for no HE images

In [13]:
less_data = data.drop(data.index[range(13,26)])

In [14]:
less_data

,paths,images,labels
0,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[27, 30, 39, 59, 180, 115, 199, 174, 124, 93,...",Grass
1,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[166, 126, 135, 143, 111, 69, 13, 25, 58, 130...",Bark
2,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[164, 188, 201, 186, 170, 157, 141, 151, 133,...",Straw
3,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[219, 192, 203, 170, 166, 133, 150, 165, 162,...",Herringbone weave
4,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[168, 157, 157, 179, 201, 195, 191, 173, 185,...",Woolen cloth
5,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[207, 88, 81, 91, 99, 163, 197, 178, 130, 131...",Pressed calf leather
6,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[161, 174, 167, 156, 191, 202, 194, 153, 137,...",Beach sand
7,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[156, 163, 168, 161, 126, 145, 167, 171, 156,...",Water
8,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[189, 199, 201, 195, 184, 199, 201, 197, 198,...",Wood grain
9,/content/drive/MyDrive/ECE594N/hw3/textures/1....,"[[150, 182, 188, 187, 184, 180, 185, 195, 168,...",Raffia


# Make train and test dataframes

In [15]:
def generate_train_test(data):
  trainData= pd.DataFrame()
  testData = pd.DataFrame() 

  testImages = []

  trainImages = []
  trainLabels = []

  for i in range(len(data)):
    img = data['images'][i]

    train1,train2,test1,test2 = divide_img(img)
    trainImages.append(train1)
    trainImages.append(train2)
    testImages.append(test1)
    testImages.append(test2)

    trainLabels.append(data['labels'][i])
    trainLabels.append(data['labels'][i])

  trainData['images']=trainImages
  trainData['labels']=trainLabels
  testData['images']=testImages
  return trainData, testData

In [16]:
less_trainData, less_testData = generate_train_test(less_data)

# generate covariance descriptors for train and test data

In [31]:
def generate_train_test_c(trainData, testData):
  # 50 128x128 window covariance descriptors for trainDataC
  # 100 128x128 window covariance descriptors for testDataC
  trainDataC = pd.DataFrame()
  testDataC = pd.DataFrame()

  # 50 128x128 window covariance descriptors for trainDataC
  trainC=[]
  trainLabelsC = []

  for j in range(len(trainData)):
    image = trainData['images'][j]
    print(j)

    for i in range(50):
      window = get_random_window(image)
      F=get_feature_space(window)
      C = get_covariance_descriptor(F)
      trainC.append(C)
      trainLabelsC.append(trainData['labels'][j])

  trainDataC['Covariance Descriptor'] = trainC
  trainDataC['labels'] = trainLabelsC

  # 100 128x128 window covariance descriptors for testDataC
  testC=[]
  testLabelsC = []

  for j in range(len(trainData)):
    image = testData['images'][j]
    print(j)

    for i in range(100):
      window = get_random_window(image)
      F=get_feature_space(window)
      C = get_covariance_descriptor(F)
      testC.append(C)
      testLabelsC.append(trainData['labels'][j])

  testDataC['Covariance Descriptor'] = testC
  testDataC['labels'] = testLabelsC

  return trainDataC, testDataC

In [ ]:
less_trainDataC, less_testDataC = generate_train_test_c(less_trainData,less_testData)

# Save Dataframes, since computing these covariances takes forever...

In [35]:
#less data, no HE images
less_data.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_data.pkl')

less_testData.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_testData.pkl')
less_trainData.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_trainData.pkl')

less_testDataC.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_testDataC.pkl')
less_trainDataC.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_trainDataC.pkl')

In [34]:
#full data, with HE images
data.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'data.pkl')

testData.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'testData.pkl')
trainData.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'trainData.pkl')

testDataC.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'testDataC.pkl')
trainDataC.to_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'trainDataC.pkl')

# Load data back into project

In [40]:
#less data, no HE images
less_data = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_data.pkl')
less_testData = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_testData.pkl')
less_trainData = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_trainData.pkl')

less_testDataC = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_testDataC.pkl')
less_trainDataC = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'less_trainDataC.pkl')

In [41]:
#full data, with HE images
data = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'data.pkl')
testData = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'testData.pkl')
trainData = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'trainData.pkl')

testDataC = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'testDataC.pkl')
trainDataC = pd.read_pickle('/content/drive/MyDrive/ECE594N/hw3/'+'trainDataC.pkl')

# define Riemannian kernel for kernel PCA using geomstats


In [42]:
from geomstats.geometry.spd_matrices import SPDMatrices
from geomstats.geometry.matrices import Matrices, MatricesMetric

In [45]:
def dist( point_a, point_b):
  """Compute log euclidean distance.

  Parameters
  ----------
  point_a : array-like, shape=[..., dim]
      Point.
  point_b : array-like, shape=[..., dim]
      Point.

  Returns
  -------
  dist : array-like, shape=[...,]
      Distance.
  """

  from numpy import linalg as LA
  log_a = SPDMatrices.logm(point_a)
  log_b = SPDMatrices.logm(point_b)
  A = log_a-log_b
  return LA.norm(A,'fro')


In [46]:
def my_kernel(X, Y,sigma = 0.1):
    """
    My Reimannian kernel:
    """
    from numpy import linalg as LA
    kr = gs.exp( (-(dist(X,Y)**2)) / (2*(sigma**2)) )
    return kr

# Use kernel PCA

## calculate gram matrix for training set

In [94]:
def generate_gram_matrix(X,Y,sigma = 0.1, kernel = my_kernel):
  '''
  generates the gram matrix for using a custom kernel with kernel pca
  X and Y are dataframes with 'Covariance Descriptor' columns
  '''
  #precomputing the gram matrix
  G = []
  for j in range(len(X['Covariance Descriptor'])):
    for i in range(len(Y['Covariance Descriptor'])):
      G.append(my_kernel(X['Covariance Descriptor'][j],Y['Covariance Descriptor'][i], sigma))

    # print(j)
    
  G = np.array(G)
  G = np.reshape(G,(X.shape[0],Y.shape[0]))
  return G


In [ ]:
less_G = generate_gram_matrix(less_trainDataC,less_trainDataC)

In [56]:
np.save('/content/drive/MyDrive/ECE594N/hw3/'+'less_G.npy',less_G)
less_G = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'less_G.npy')

### save/load gram matrix

In [ ]:
np.save('/content/drive/MyDrive/ECE594N/hw3/'+'G.npy',G)

In [ ]:
G = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G.npy')

## Use kernel PCA with precomputed gram matrix for training

In [ ]:
from sklearn.decomposition import KernelPCA

transformer = KernelPCA(n_components=10, kernel='precomputed')
X_transformed = transformer.fit_transform(G)
X_transformed.shape

(2600, 10)

In [ ]:
X_transformed

array([[-0.03417568,  0.00945475, -0.08399346, ..., -0.32723283,
        -0.55862632,  0.41073195],
       [-0.03533698,  0.00978737, -0.08696432, ..., -0.33685276,
        -0.57350083,  0.41983963],
       [-0.03038842,  0.00829328, -0.07325832, ..., -0.2754946 ,
        -0.46795075,  0.34215576],
       ...,
       [-0.02041733,  0.00484138, -0.03959354, ..., -0.02871516,
         0.00257827, -0.05597037],
       [-0.02015779,  0.00477368, -0.03901905, ..., -0.02820234,
         0.00253089, -0.05490564],
       [-0.01878257,  0.00440835, -0.03589424, ..., -0.02524323,
         0.00225444, -0.04860936]])

In [ ]:
np.save('/content/drive/MyDrive/ECE594N/hw3/'+'X_transformed.npy',X_transformed)

## calculate gram matrix for test set

In [ ]:
testDataC

In [ ]:
#precomputing the gram matrix
Gtest = []
for j in range(len(testDataC['Covariance Descriptor'])):
  for i in range(len(trainDataC['Covariance Descriptor'])):
    Gtest.append(my_kernel(testDataC['Covariance Descriptor'][j],trainDataC['Covariance Descriptor'][i]))

  print(j)

In [ ]:
Gtest = np.array(Gtest)
Gtest = np.reshape(Gtest,(5200,2600))

In [ ]:
Gtest.shape

(5200, 2600)

In [ ]:
np.save('/content/drive/MyDrive/ECE594N/hw3/'+'Gtest.npy',Gtest)

In [ ]:
Gtest = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'Gtest.npy')

## use kernel pca for test

In [ ]:
transformer = KernelPCA(n_components=10, kernel='precomputed')

In [ ]:
Xtest_transformed = transformer.transform(Gtest)
Xtest_transformed.shape

(5200, 10)

In [ ]:
Xtest_transformed

array([[-0.01303235,  0.0029131 , -0.02326809, ..., -0.02147553,
        -0.01682494, -0.00557485],
       [-0.01262491,  0.00279137, -0.02215628, ..., -0.01662819,
        -0.00851234, -0.01167648],
       [-0.01308604,  0.00293029, -0.02342984, ..., -0.02235992,
        -0.01840967, -0.0043423 ],
       ...,
       [-0.01711853,  0.00396499, -0.0320968 , ..., -0.02160531,
         0.00191351, -0.04081956],
       [-0.01479652,  0.00333174, -0.02661586, ..., -0.01589901,
         0.00137004, -0.02812595],
       [-0.01552616,  0.0035198 , -0.02820271, ..., -0.0172423 ,
         0.00149322, -0.03084748]])

# Implement knn

## checking data on itself

In [114]:
from sklearn.neighbors import KNeighborsClassifier
#convert dataframe labels to list of targets
ytrain = trainDataC['labels'].values.tolist()

neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(X_transformed, ytrain)

yhat = neigh.predict(X_transformed)

print(accuracy_score(ytrain,yhat))
cm = confusion_matrix(ytrain, yhat)
print(cm)

0.7707692307692308
[[161   0   6   0   0   0  10   0   6   1   6  10   0]
 [  0 196   0   0   0   0   0   0   0   0   0   4   0]
 [ 20   0  84   0   0  14   9   0   0  46  22   5   0]
 [  0   0   0 200   0   0   0   0   0   0   0   0   0]
 [  2   0   0   0 169   0   0   0   2   0   0  27   0]
 [  3   0  21   0   0 161   4   0   0   0   9   2   0]
 [ 22   0  12   0   0   9 137   0   4   6   7   3   0]
 [  0   0   0   0   0   0   0 200   0   0   0   0   0]
 [  0   0   0   0   0   1  23   0 175   0   1   0   0]
 [  0   0  31   0   0   0   0   0   0 141  27   1   0]
 [  6   0  26   0   0  17   3   0   0  39 105   4   0]
 [ 25   0  11   0  36   0   8   0   3  29  13  75   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 200]]


## predict labels for test data

In [115]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

yhat = neigh.predict(Xtest_transformed)

#real labels
ytest = testDataC['labels'].values.tolist()

#accuracy
print(accuracy_score(ytest,yhat))
cm = confusion_matrix(ytest, yhat)
print(cm)

0.3151923076923077
[[ 90   0  68   0   7   0  48   0  91  34  20  42   0]
 [  0 393   0   0   0   0   0   0   0   6   0   1   0]
 [  0   0   2   0   0   0   0   0   0 398   0   0   0]
 [  0  83   0   3   0   0   0   0   0 314   0   0   0]
 [  1   6  12   0 279   0   6   0  44  11   0  41   0]
 [ 29   0  53   0   0 150  36   0   0 105   3  24   0]
 [ 50   0 142   0   0  10  74   0   0  55  10  59   0]
 [  0   0   0  33   0   0   0 197   0 170   0   0   0]
 [  3   0  52   0   0   0  11   0  14 306   1  13   0]
 [  0   0  17   0   0   0   0   0   0 382   1   0   0]
 [  2   0 111   0   0   2  35   0   0 206  25  19   0]
 [  5   0  78   0  13   0  10   0   6 255   8  25   0]
 [  0   0  12   0   0   0   0   0   0 383   0   0   5]]


## Using no HE images

In [65]:
from sklearn.decomposition import KernelPCA
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier

transformer = KernelPCA(n_components=10, kernel='precomputed')
X_transformed = transformer.fit_transform(less_G)
print(X_transformed.shape)

#convert dataframe labels to list of targets
ytrain = less_trainDataC['labels'].values.tolist()

neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_transformed, ytrain)

yhat = neigh.predict(X_transformed)

print(accuracy_score(ytrain,yhat))
cm = confusion_matrix(ytrain, yhat)
print(cm)

(1300, 10)
0.9061538461538462
[[100   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  77   0   0   0   0   0   0  18   5   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 100   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 100   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  99   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0 100   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0   0   0]
 [  0   0  19   0   0   0   0   0   0  69  12   0   0]
 [  0   0  17   0   0   0   0   0   0  14  57  12   0]
 [  0   0   3   0   0   0   0   0   0   7  14  76   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 100]]


#find best value for sigma

In [70]:
sigmas = [0.1,0.2,0.4,0.8,1]

for s in sigmas: 
  print('sigma is ',s,'\n')

  G_sigma = generate_gram_matrix(less_trainDataC,less_trainDataC,sigma=s)

  np.save('/content/drive/MyDrive/ECE594N/hw3/'+'G_sigma'+str(s)+'.npy',G_sigma)
  # less_G = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G_sigma'+str(s)+'.npy')


  transformer = KernelPCA(n_components=10, kernel='precomputed')
  X_transformed = transformer.fit_transform(G_sigma)
  print(X_transformed.shape)

  #convert dataframe labels to list of targets
  ytrain = less_trainDataC['labels'].values.tolist()

  neigh = KNeighborsClassifier(n_neighbors=5)
  neigh.fit(X_transformed, ytrain)

  yhat = neigh.predict(X_transformed)

  print('The accuracy with sigma = ',s, 'is',accuracy_score(ytrain,yhat),'\n')
  cm = confusion_matrix(ytrain, yhat)
  print(cm)
  print('********************')

sigma is  0.1 

(1300, 10)
The accuracy with sigma =  0.1 is 0.9061538461538462 

[[100   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0  77   0   0   0   0   0   0  18   5   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 100   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 100   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0  99   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0 100   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0   0   0]
 [  0   0  19   0   0   0   0   0   0  69  12   0   0]
 [  0   0  17   0   0   0   0   0   0  14  57  12   0]
 [  0   0   3   0   0   0   0   0   0   7  14  76   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 100]]
********************
sigma is  0.2 

(1300, 10)
The accuracy with sigma =  0.2 is 0.9907692307692307 

[[100   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0   0   0   

# using smallest good sigma value, sigma = 0.4, varying L, Riemannian Metric

In [236]:
ls = [10,11,12,15]
for l in ls:
  print(l)
  s = .4
  #load best sigma for gram(X,X)
  G_fit = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G_sigma'+str(s)+'.npy')

  #fit kernel pca
  transformer = KernelPCA(n_components=l, kernel='precomputed')
  X_transformed = transformer.fit_transform(G_fit)
  print(X_transformed.shape)

  #convert dataframe labels to list of targets
  ytrain = less_trainDataC['labels'].values.tolist()

  neigh = KNeighborsClassifier(n_neighbors=5)
  neigh.fit(X_transformed, ytrain)

  yhat = neigh.predict(X_transformed)

  print(accuracy_score(ytrain,yhat))
  cm = confusion_matrix(ytrain, yhat)
  print(cm)

  #generate gram matrix for test data
  print('***********************')
  print('using test data with l = ',l)
  # G_test = generate_gram_matrix(less_testDataC,less_trainDataC)
  G_test = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G_test.npy')


  Xtest_transformed = transformer.transform(G_test)
  print('Xtest_transformed is shape ',Xtest_transformed.shape)

  #convert dataframe labels to list of targets
  ytest = less_testDataC['labels'].values.tolist()

#   neigh = KNeighborsClassifier(n_neighbors=5)
#   neigh.fit(Xtest_transformed, ytest)

  yhat = neigh.predict(Xtest_transformed)

  print(accuracy_score(ytest, yhat))
  cm = confusion_matrix(ytest, yhat)
  print(cm)
  print('*********************************************************************')

10
(1300, 10)
1.0
[[100   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 100   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0 100   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0 100   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0 100   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0 100   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0 100   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0 100   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0 100   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0 100   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0 100   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0 100]]
***********************
using test data with l =  10
Xtest_transformed is shape  (2600, 10)
0.9857692307692307
[[200   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0 199   0   0   0   1   0   0   0   0   0   0   0]
 [  0   0 197   0   0   0   0   0   0   3   0

#find best sigma for euclidean metric

In [ ]:
sigmas = [0.1,0.2,0.4,0.8,1]

for s in sigmas: 
  print('sigma is ',s,'\n')

  G_sigma = generate_gram_matrix(less_trainDataC,less_trainDataC,sigma=s)

  np.save('/content/drive/MyDrive/ECE594N/hw3/'+'G_sigma'+str(s)+'.npy',G_sigma)
  # less_G = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G_sigma'+str(s)+'.npy')


  transformer = KernelPCA(n_components=10, kernel='precomputed')
  X_transformed = transformer.fit_transform(G_sigma)
  print(X_transformed.shape)

  #convert dataframe labels to list of targets
  ytrain = less_trainDataC['labels'].values.tolist()

  neigh = KNeighborsClassifier(n_neighbors=5)
  neigh.fit(X_transformed, ytrain)

  yhat = neigh.predict(X_transformed)

  print('The accuracy with sigma = ',s, 'is',accuracy_score(ytrain,yhat),'\n')
  cm = confusion_matrix(ytrain, yhat)
  print(cm)
  print('********************')

# using smallest good sigma value, sigma = 0.4, varying L, Euclidean Metric

In [134]:
less_trainDataC['Covariance Descriptor'][0].flatten()

array([ 3071.54866701,  -103.38451442,  -144.30252444, -3247.9420758 ,
       -2498.17334023,  -103.38451442,  7278.38281417,    39.15689772,
        1147.01569188,   157.22098374,  -144.30252444,    39.15689772,
        5098.39688419,   221.38662067,  1057.89283258, -3247.9420758 ,
        1147.01569188,   221.38662067, 11916.69841522,  1460.82044017,
       -2498.17334023,   157.22098374,  1057.89283258,  1460.82044017,
       10194.1671617 ])

In [204]:
xtrain = []
for i in range(len(less_trainDataC)):
  xtrain.append(less_trainDataC['Covariance Descriptor'][i].flatten())

xtrain = np.array(xtrain)
xtrain.shape

(1300, 25)

In [203]:
xtest = []
for i in range(len(less_testDataC)):
  xtest.append(less_testDataC['Covariance Descriptor'][i].flatten())

xtest = np.array(xtest)
xtest.shape

(2600, 25)

In [ ]:
from sklearn.metrics.pairwise import pairwise_kernels
pairwise_kernels(xtrain[0], xtrain[0], metric='linear')

In [196]:
def generate_gram_matrix_euc(X,Y):
  '''
  generates the gram matrix for using a euclidean kernel with kernel pca
  X and Y are vectors
  '''

  #precomputing the gram matrix
  G = []
  for j in range(len(X)):
    for i in range(len(Y)):
      # G.append(X[i]@Y[i].T)
      G.append(np.linalg.norm(X[j]-Y[i]))

    # print(j)
    
  G = np.array(G)
  G = np.reshape(G,(X.shape[0],Y.shape[0]))
  return G

In [197]:
Geuc_train = generate_gram_matrix_euc(xtrain,xtrain)

In [198]:
Geuc_test = generate_gram_matrix_euc(xtest,xtrain)

In [191]:
def generate_gram_matrix_euc2(X,Y,sigma = 0.1):
  '''
  generates the gram matrix for using a custom kernel with kernel pca
  X and Y are dataframes with 'Covariance Descriptor' columns
  '''
  #precomputing the gram matrix
  G = []
  for j in range(len(X['Covariance Descriptor'])):
    for i in range(len(Y['Covariance Descriptor'])):
      G.append(gs.exp( (-((np.linalg.norm(X['Covariance Descriptor'][j]-Y['Covariance Descriptor'][i]) ) **2)) / (2*(sigma**2))))

    # print(j)
    
  G = np.array(G)
  G = np.reshape(G,(X.shape[0],Y.shape[0]))
  return G

In [230]:
Geuc_train = generate_gram_matrix_euc2(less_trainDataC,less_trainDataC)

In [231]:
Geuc_test = generate_gram_matrix_euc2(less_testDataC,less_trainDataC)

In [235]:
ls = [10,11,12,15]
from sklearn.decomposition import PCA
for l in ls:
  print(l)
  s = 0.4
  #load best sigma for gram(X,X)
  G_fit = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G_sigma'+str(s)+'.npy')

  #fit kernel pca
  transformer = KernelPCA(n_components=l,kernel='precomputed')
  X_transformed = transformer.fit_transform(Geuc_train)
  print(X_transformed.shape)

  #convert dataframe labels to list of targets
  ytrain = less_trainDataC['labels'].values.tolist()

  neigh = KNeighborsClassifier(n_neighbors=5)
  neigh.fit(X_transformed, ytrain)

  yhat = neigh.predict(X_transformed)

  print(accuracy_score(ytrain,yhat))
  cm = confusion_matrix(ytrain, yhat)
  print(cm)

  #generate gram matrix for test data
  print('***********************')
  print('using test data with l = ',l)
  # G_test = generate_gram_matrix(less_testDataC,less_trainDataC)
  # G_test = np.load('/content/drive/MyDrive/ECE594N/hw3/'+'G_test.npy')


  Xtest_transformed = transformer.transform(Geuc_test)

  print('Xtest_transformed is shape ',Xtest_transformed.shape)

  #convert dataframe labels to list of targets
  ytest = less_testDataC['labels'].values.tolist()

#   neigh = KNeighborsClassifier(n_neighbors=5)
#   neigh.fit(Xtest_transformed, ytest)

  yhat = neigh.predict(Xtest_transformed)

  print(accuracy_score(ytest, yhat))
  cm = confusion_matrix(ytest, yhat)
  print(cm)
  print('*********************************************************************')

10
(1300, 10)
0.07692307692307693
[[  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]
 [  0   0   0 100   0   0   0   0   0   0   0   0   0]]
***********************
using test data with l =  10
Xtest_transformed is shape  (2600, 10)
0.07692307692307693
[[  0   0   0 200   0   0   0   0   0   0   0   0   0]
 [  0   0   0 200   0   0   0   0   0   0   0   0   0]
 [  0   0   0 200   0   0   